In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [2]:
!pip install import-ipynb

In [3]:
import import_ipynb

In [4]:
%cd /content/drive/MyDrive/A62_projet_synthese_feature/notebook

/content/drive/MyDrive/A62_projet_synthese_feature/notebook


In [5]:
%run iteration_01_0_imports.py

In [6]:
%run iteration_01_0_parameters.py

In [7]:
import iteration_01_0_utils as utils

importing Jupyter notebook from iteration_01_0_utils.ipynb


In [8]:
%whos

Variable           Type      Data/Info
--------------------------------------
HEALTHY            str       H
IMG_SIZE           int       400
Image              module    <module 'PIL.Image' from <...>t-packages/PIL/Image.py'>
NBR_SAMPLE         int       20
PATH_DATA          str       ../data
PATH_DATA_EDA      str       ../data/eda
PATH_DATA_LIST     str       ../data/list
PATH_DATA_SAMPLE   str       ../data/sample
PATH_MODEL         str       ../model
PATH_ROOT          str       ..
PATH_SOURCE_IMG    str       /content/drive/MyDrive/A6<...>_synthese_lnk/cell_images
SICK               str       S
cv2                module    <module 'cv2.cv2' from '/<...>37m-x86_64-linux-gnu.so'>
datetime           type      <class 'datetime.datetime'>
drive              module    <module 'google.colab.dri<...>s/google/colab/drive.py'>
glob               module    <module 'glob' from '/usr/lib/python3.7/glob.py'>
import_ipynb       module    <module 'import_ipynb' fr<...>ackages/import_ipynb.py'>


In [9]:
df_path_img_sick = pd.read_csv(PATH_DATA_LIST + "/" + "df_list_path_sick_img.csv")
df_path_img_healthy = pd.read_csv(PATH_DATA_LIST + "/" + "df_list_path_healthy_img.csv")
df_path_img = pd.read_csv(PATH_DATA_LIST + "/" + "df_list_path_img.csv")

# Prétraitement

## positif_global_std

In [10]:
IMG_IN_COLOR = 1
def positive_global_std(path_img: str) -> np.array:
    img_arr = cv2.imread(path_img, flags = IMG_IN_COLOR)
    mean = img_arr.mean()
    std  = img_arr.std()
    img_arr_std = (img_arr - mean) / std
    
    # clip les valeurs
    img_arr_std = np.clip(img_arr_std, -1.0, 1.0)
    
    # shift to 0--1
    img_arr_std = (img_arr_std + 1.0) / 2.0
    
    return img_arr_std

* Voir avec le prof la pertinence de clip les valeurs extremes -1 et 1

In [11]:
NBR_SAMPLE

20

In [12]:
df_path_img_healthy_sample = df_path_img_healthy.sample(n = int(NBR_SAMPLE/2))
df_path_img_sick_sample    = df_path_img_sick.sample(n = int(NBR_SAMPLE/2))

In [13]:
PATH_DATA_SAMPLE

'../data/sample'

In [14]:
DATE_TIME = "2021-11-18_20-39-00"

def create_sample(df_path_img:str, img_type:str, path_sample:str, date_time:str, clean_dir:bool):
    path_sample_time = path_sample+"/"+date_time

    os.makedirs(path_sample_time, exist_ok=True)
    if clean_dir:
        file_list = glob.glob(path_sample_time+"/"+"*.*")
        for f in file_list:
            os.remove(f)
    
    for path_img , _ in zip(df_path_img.path_img, tqdm(range(len(df_path_img.path_img)))):
        
        _, img_name, _ = utils.split_file_info(path_img)
        
        img_arr = positive_global_std(path_img)
        # padding
        img_arr_padded = np.zeros((IMG_SIZE, IMG_SIZE, 3))
        img_arr_padded[:img_arr.shape[0],:img_arr.shape[1],:] = img_arr
        
        utils.pickel_save(img_arr_padded, path_sample_time + "/" + str(img_type).upper() + "_" + img_name + ".pkl")
        

In [15]:
create_sample(df_path_img_healthy_sample, HEALTHY, PATH_DATA_SAMPLE, DATE_TIME, clean_dir=True)

 90%|█████████ | 9/10 [00:00<00:00, 25.06it/s]


In [16]:
create_sample(df_path_img_sick_sample, SICK, PATH_DATA_SAMPLE, DATE_TIME, clean_dir=False)

 90%|█████████ | 9/10 [00:00<00:00, 31.52it/s]


In [17]:
df_path_img_sample = pd.concat([df_path_img_healthy_sample, df_path_img_sick_sample], ignore_index=True)
df_path_img_not_in_sample = pd.DataFrame([i for i in df_path_img.path_img.values if not i in df_path_img_sample.path_img.values], 
                 columns=df_path_img_sample.columns)

In [18]:
df_path_img_not_in_sample.shape

(30, 1)

In [19]:
df_path_img_not_in_sample.head()

,path_img
0,/content/drive/MyDrive/A62_projet_synthese_dev...
1,/content/drive/MyDrive/A62_projet_synthese_dev...
2,/content/drive/MyDrive/A62_projet_synthese_dev...
3,/content/drive/MyDrive/A62_projet_synthese_dev...
4,/content/drive/MyDrive/A62_projet_synthese_dev...


In [20]:
df_path_img_not_in_sample.sample(1).values

array([['/content/drive/MyDrive/A62_projet_synthese_develop_main/A62_projet_synthese_lnk/cell_images/Parasitized/C33P1thinF_IMG_20150619_121503a_cell_158.png']],
      dtype=object)